In [1]:
from math import ceil, floor
from itertools import permutations

In [2]:
class Node:
    def __init__(self, chunk='', parent=None):
        if not isinstance(chunk, str):
            chunk = str(chunk)
        self.left = self.right = self.value = None
        self.parent = parent
        depth = 0
        for index, char in enumerate(chunk[1:-1]):
            if char == '[':
                depth += 1
            elif char == ']':
                depth -= 1
            elif char == ',' and depth == 0:
                self.left = Node(chunk[1:index+1], parent=self)
                self.right = Node(chunk[index+2:-1], parent=self)
                break
        if chunk.isnumeric() and not self.has_children:
            self.value = int(chunk)
    
    def __str__(self):
        if not self.has_children:
            return str(self.value)
        return f"[{self.left},{self.right}]"
    
    def __repr__(self):
        return str(self)
    
    def __add__(self, node):
        new = Node()
        new.left = Node(self, parent=new)
        new.right = Node(node, parent=new)
        while new.explode() or new.split():
            continue
        return new
    
    def __radd__(self, val):
        if val == 0:
            return self
        return self.__add__(val)
    
    @property
    def has_children(self):
        return self.left and self.right
    
    def split(self):
        if self.value and self.value > 9:
            self.left = Node(floor(self.value / 2), parent=self)
            self.right = Node(ceil(self.value / 2), parent=self)
            self.value = None
            return True
        if self.has_children:
            return self.left.split() or self.right.split()
        return False
    
    def explode(self, depth=0):
        if self.has_children:
            if depth == 4 and not self.left.has_children and not self.right.has_children:
                # Left
                left, previous = self.parent, self
                while left and left.left == previous:
                    previous = left
                    left = left.parent
                if left:
                    left = left.left
                while left and left.right:
                    left = left.right
                # Right
                right, previous = self.parent, self
                while right and right.right == previous:
                    previous = right
                    right = right.parent
                if right:
                    right = right.right
                while right and right.left:
                    right = right.left
                # Sum
                if left:
                    left.value += self.left.value
                if right:
                    right.value += self.right.value
                self.left = self.right = None
                self.value = 0
                return True
            return self.left.explode(depth + 1) or self.right.explode(depth + 1)
        return False
    
    @property
    def magnitude(self):
        if self.has_children:
            return 3 * self.left.magnitude + 2 * self.right.magnitude
        return self.value

In [3]:
with open('Day18.txt') as file:
    nodes = [Node(line) for line in file.read().splitlines()]

In [4]:
%%time
sum(nodes).magnitude

CPU times: user 160 ms, sys: 1.72 ms, total: 162 ms
Wall time: 160 ms


4033

In [5]:
%%time
max((a + b).magnitude for a, b in permutations(nodes, 2))

CPU times: user 2.73 s, sys: 2.02 ms, total: 2.74 s
Wall time: 2.74 s


4864